In [1]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy

from statsmodels.sandbox.regression.gmm import GMM
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge

In [2]:
data = pd.read_stata('chile.dta')
data = data.set_index(['id','year'])

In [3]:
ldata = data.copy()

ldata[['routput', 'totlab', 'renerg','realmats','rcapstock']] = np.log(
    ldata[['routput','totlab', 'renerg','realmats','rcapstock']])

ldata = ldata.dropna()

do a non-parametric regression with $l_{it}$, $m_{it}$, $k_{it}$ on $y_{it}$ and save residual, and fitted value

given a guess of $\beta_1$ and $\beta_2$ compute an implied omega i.e. productive shock
get wit-1 (i.e. without estimating a function)
the residuals should be exogenous
these are the moment conditions

https://www.aeaweb.org/content/file?id=3015

In [5]:
d = ldata['routput'],ldata[['totlab']],ldata[['rcapstock','realmats','renerg']]

def np_resids(y,x):
    """residuals from lasso"""
    poly = PolynomialFeatures(degree=3)
    x_poly = poly.fit_transform(x)
    clf = Ridge(alpha=0.0)
    clf.fit(x_poly, y) 
    resid = y-clf.predict(x_poly)
    return resid


stage1_resids = np_resids(ldata['routput'],ldata[['totlab','rcapstock','renerg','realmats']])

In [29]:
def stage2obj(params, stage1_resids, dta):
    y,x = dta
    rho = params[:-1]
    params = params[:-1]
    
    x = sm.add_constant(x)
    xlag= x.groupby('id').shift(1)
    
    #compute the value of the control function from stage 1
    control = y - stage1_resids
    controllag = control.groupby('id').shift(1)
    
    print 'yo'
    print rho
    ### compute the value of the shock implied by the control###
    shock = (y - x.mul(params,axis=1).sum(axis=1) 
             - rho*(controllag- xlag.mul(params,axis=1).sum(axis=1) ) )    
    
    #####multiply to get moment condition #########
    moments = x.mul(shock,axis=0)
    return moments.dropna()
    
d = ldata['routput'],ldata[['rcapstock','totlab','renerg']]
stage2obj(np.ones(5), stage1_resids, d)

yo
[1. 1. 1. 1.]


ValueError: operands could not be broadcast together with shapes (20717,) (4,) 

In [19]:
class ACF(GMM):
    
    def __init__(self, dta, stage1_resid, *args, **kwds):
        # set appropriate counts for moment conditions and parameters
        y,x = dta
        super(ACF, self).__init__(y,x,stage1_resid, x.shape[1],*args, **kwds)
        self.endog = y
        self.exog = x
        self.instr = stage1_resids
        
    def momcond(self, params):
        d = self.endog, self.exog
        return np.array( stage2obj(params, self.instr , d) )
        
        
d = ldata['routput'],ldata[['rcapstock','totlab','renerg']]
acfmodel = ACF(d, stage1_resids)
acfresult = acfmodel.fit(np.array([.1,.1,.1]),optim_method='nm')
print acfresult.summary()

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 87
         Function evaluations: 158
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 34
         Function evaluations: 60
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 34
         Function evaluations: 60
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 34
         Function evaluations: 60
                                 ACF Results                                  
Dep. Variable:                routput   Hansen J:                    6.799e-06
Model:                            ACF   Prob (Hansen J):                   nan
Method:                           GMM                                         
Date:                Wed, 15 May 2019                                         
Time:                        12:20:33               